# 3. Naiver Bayesklassifikator zur Gesichtserkennung

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
%matplotlib inline
%load_ext version_information

Implementieren Sie den Gaussian-Naïve-Bayes-Klassifikator aus der Vorlesung. Testen Sie Ihre Implementierung am Datensatz ''Labeled Faces in the Wild'' aus Aufgabe 2, wiederum nur für Personen, für die mindestens 70 Bilder existieren. Teilen Sie Ihren Datensatz in 60 % Trainings- und 40% Testdaten (nach vorheriger Zufalls-Permutation der Reihenfolge) und skalieren Sie die Bilder wieder auf 1/8 der Originalgröße. Führen Sie anschließend eine Hauptkomponentenanalyse auf den Trainingsdaten durch und projizieren Sie sowohl Trainings- als auch Testbilder auf die ersten 7 Eigengesichter. Trainieren Sie Ihren GNB-Klassifikator auf dem Trainingsdatensatz als ''George-W.-Bush-Detektor'', d.h. alle zu dieser Person gehörigen Bilder werden mit 1 gelabelt, alle sonstigen mit –1. Werten Sie Ihren Klassifikator sowohl auf den Trainings- wie auf den unabhängigen Testdaten aus. Bestimmen Sie dafür jeweils die Detektionswahrscheinlichkeit, Richtig-Negativ-Rate, Fehlalarmrate und Falsch-Negativ-Rate.

In [2]:
from scipy.stats import norm

def GaussianNaiveBayes(X, y):
    classes = np.unique(y)
    mean = {}
    std = {}
    class_prior = {}

    # Calculate mean and standard deviation for each feature in each class
    for c in classes:
        X_c = X[y == c]
        mean[c] = np.mean(X_c, axis=0)
        std[c] = np.std(X_c, axis=0)

        # Calculate class prior probability
        class_prior[c] = len(X_c) / len(X)

    predictions = []

    # For every feature
    for x in X:
        posteriors = []

        # Calculate posterior probability for each class
        for c in classes:
            prior = np.log(class_prior[c])
            likelihood = np.sum(np.log(norm.pdf(x, loc=mean[c], scale=std[c])))
            posterior = prior + likelihood
            posteriors.append(posterior)

        # Append the class with the highest posterior probability to predictions
        predictions.append(classes[np.argmax(posteriors)])

    return np.array(predictions)

In [ ]:
import os
import tarfile
from urllib.request import urlretrieve

lfw_filename = 'lfw-funneled.tgz'
lfw_directory = '.lfw-dataset'

if not os.path.isfile(lfw_filename):
    print("Downloading")
    urlretrieve('http://vis-www.cs.umass.edu/lfw/lfw-funneled.tgz',filename = lfw_filename)

if not os.path.isdir(lfw_directory):
    # Dateien in das Zielverzeichnis extrahieren
    with tarfile.open(lfw_filename, 'r:gz') as tar:
        tar.extractall(path=lfw_directory)

min_images_required = 70
selected_persons = []
extracted_path = ".lfw-dataset/lfw_funneled"

# Verzeichnis durchsuchen
for person_folder in os.listdir(extracted_path):
    person_path = os.path.join(extracted_path, person_folder)

    if os.path.isdir(person_path):
        # Anzahl der Bilder fuer die aktuelle Person zaehlen
        num_images = len([f for f in os.listdir(person_path) if f.endswith('.jpg')])

        # Ueberpruefen, ob Mindestanzahl erreicht
        if num_images >= min_images_required:
            selected_persons.append({
                'person_name': person_folder,
                'num_images': num_images
            })

print(f"Personen mit mindestens {min_images_required} Bildern:")
for person_info in selected_persons:
    print(f"{person_info['person_name']}: {person_info['num_images']} Bilder")

In [ ]:
from skimage import io, transform
dataset_path    = extracted_path
train_data_path = ".lfw-dataset-train_2"
test_data_path  = ".lfw-dataset-test_2"
os.makedirs(train_data_path, exist_ok=True)
os.makedirs(test_data_path, exist_ok=True)

# Daten verarbeiten
for person in selected_persons:
    person_folder = person['person_name']
    person_path = os.path.join(dataset_path, person_folder)
    all_person_data = []
    train_person_data = []
    test_person_data = []
    first = True

    # Bilder laden
    for filename in os.listdir(person_path):
        if filename.endswith('.jpg'):
            image_path = os.path.join(person_path, filename)
            # Laden und in Graustufen konvertieren
            img = io.imread(image_path, as_gray=True)
            # Skalieren auf 32x32
            img = transform.resize(img, (32, 32))
            # In einen Vektor packen
            img_vector = img.flatten()
            all_person_data.append(img_vector)

    # Trainingsdaten in einer gemeinsamen Designmatrix speichern
    train_person_data = np.array(train_person_data)
    np.save(os.path.join(train_data_path, f"{person_folder}_train.npy"), train_person_data)

    # Testdaten getrennt abspeichern
    test_person_data = np.array(test_person_data)
    np.save(os.path.join(test_data_path, f"{person_folder}_test.npy"), test_person_data)

In [ ]:
%version_information